# Stacking（堆叠集成）详解

**Stacking**（Stacked Generalization，堆叠泛化）是集成学习（Ensemble Learning）的一种高级方法。
它通过**组合多个不同模型（基学习器）的预测结果**，再使用一个次级模型（元学习器）进行最终预测。

---

## 一、核心思想

与Bagging和Boosting不同，Stacking不是简单地投票或平均，而是：
1. **先训练一组多样化的基学习器**
2. **再用它们的输出作为新特征**
3. **训练一个次级学习器进行最终预测**

这种方法能有效利用多个模型的不同优势。

---

## 二、算法流程

以分类任务为例，简要步骤如下：

1. **训练基学习器**
   - 在训练集上拟合多个不同的模型（如逻辑回归、决策树、SVM）

2. **生成元特征**
   - 用每个基模型对样本预测（通常是预测概率）
   - 将所有基模型的预测拼接成新特征矩阵

3. **训练元学习器**
   - 以基学习器的输出作为输入特征
   - 以原始标签作为目标
   - 训练一个新的模型（如逻辑回归、线性回归）

4. **预测**
   - 基学习器先对测试集生成预测
   - 元学习器使用这些预测输出最终结果

---

## 三、示意图

```
原始特征
     |
-------------------------
|    |    |    |        |
模型1 模型2 模型3 ... 模型N (基学习器)
|     |     |           |
预测1 预测2 预测3      预测N
 \     \     \          /
  \     \     \        /
   ------拼接为新特征-------
               |
           元学习器
               |
          最终预测
```

---

## 四、Stacking与Voting区别

| 特性             | Voting                | Stacking               |
|------------------|-----------------------|------------------------|
| 组合方式         | 投票或平均            | 元模型学习组合         |
| 是否可学习权重   | 否（或简单权重）      | 是（通过元学习器）     |
| 泛化能力         | 一般                 | 更强                  |
| 实现难度         | 简单                 | 中等                  |

---

## 五、优缺点

✅ **优点**
- 能充分利用不同模型的互补性
- 提高预测准确率
- 灵活：基模型、元模型均可选

❌ **缺点**
- 实现比Bagging/Boosting复杂
- 训练/预测速度慢
- 容易过拟合（需交叉验证）

---

## 六、Python示例（scikit-learn）

这里演示一个二分类任务：

```python
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import accuracy_score

# 加载数据
X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# 定义基模型
base_learners = [
    ('lr', LogisticRegression(max_iter=200)),
    ('dt', DecisionTreeClassifier(max_depth=3)),
    ('knn', KNeighborsClassifier(n_neighbors=5))
]

# 定义元模型
meta_learner = LogisticRegression()

# 构建Stacking
stacking = StackingClassifier(
    estimators=base_learners,
    final_estimator=meta_learner,
    passthrough=False,  # 如果为True，会把原始特征和预测拼接
    cv=5
)

stacking.fit(X_train, y_train)
y_pred = stacking.predict(X_test)

print("Stacking Accuracy:", accuracy_score(y_test, y_pred))
```

---

## 七、实用技巧

- **交叉验证**：生成元特征时要用交叉验证避免信息泄漏
- **多样性**：基学习器应尽可能不同（模型、超参）
- **元学习器**：通常用简单模型（如逻辑回归）
- **调参**：需对基模型与元模型分别调参

---

## 八、常见应用

- 比赛（Kaggle、天池）
- 推荐系统
- 金融风控

---

## 九、与其他集成对比

| 方法            | 样本采样       | 特征采样       | 学习器依赖性     | 权重学习      |
|-----------------|----------------|----------------|------------------|---------------|
| Bagging         | 有（Bootstrap）| 无             | 无（并行）       | 无            |
| Boosting        | 有（重加权）   | 无             | 有（串行）       | 有            |
| Stacking        | 无              | 无             | 无（并行）       | 有（元学习器）|

---

## 十、总结

**Stacking**：
- 将不同模型的预测融合
- 可显著提升性能
- 是机器学习集成最强方法之一

> **提示**：如果你有多种模型且它们表现相近，Stacking非常值得尝试。

---

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from mlxtend.classifier import StackingClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np

In [4]:
data = np.loadtxt('data/wine.data', delimiter=',')
X = data[:, 1:]
y = data[:, 0:1]
X_train, X_test, y_train, y_test = train_test_split(X, y.ravel(), train_size=0.8, random_state=0)


In [5]:
clf1 = KNeighborsClassifier(n_neighbors=5)
clf2 = RandomForestClassifier(n_estimators=500, random_state=1)
clf3 = GaussianNB()


In [6]:
lr = LogisticRegression()

In [7]:
sclf = StackingClassifier(classifiers=[clf1, clf2, clf3], meta_classifier=lr)

In [8]:
for model in (clf1, clf2, clf3, sclf):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(model.__class__.__name__, accuracy_score(y_test, y_pred))

KNeighborsClassifier 0.8055555555555556
RandomForestClassifier 0.9722222222222222
GaussianNB 0.9166666666666666
StackingClassifier 0.9722222222222222
